# NN 2.0

## Defining data functions

Import needed libraries

In [1]:
# imports
import pandas as pd
import tensorflow as tf
import tensorflow.keras as ks
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

Get data from file and remove columns with text

In [2]:
def get_swissvotes_data(now:int=646, since:int=299)->pd.DataFrame:
    import re
    
    dataset = pd.read_csv("../data/formatted/swissvotes_dataset_after_1900_utf8.csv", sep=';')
    
    regex = re.compile("pdev_.*")
    to_excl = list(filter(regex.match, dataset.columns))
    
    dataset.drop(columns=to_excl, inplace=True)
    dataset.drop(columns=["legisjahr"], inplace=True)
    dataset.drop(columns=["titel_kurz_d", "titel_kurz_f", "titel_off_d", "titel_off_f", "stichwort"], inplace=True)
    dataset.drop(columns=["swissvoteslink", "anzahl", "anneepolitique", "bkchrono_de", "bkchrono_fr"], inplace=True)
    dataset.drop(columns=["curiavista_de", "curiavista_fr", "urheber", "bkresults_de", "bkresults_fr"], inplace=True)
    dataset.drop(columns=["bfsmap_de", "bfsmap_fr", "nach_cockpit_d", "nach_cockpit_f", "nach_cockpit_e"], inplace=True)
    dataset = dataset[dataset["anr"] < now] # we don't care about future votes
    dataset = dataset[dataset["anr"] >= since]
    
    return dataset
print(f"Defined {get_swissvotes_data}")

Defined <function get_swissvotes_data at 0x000002340DCF1EE0>


In [3]:
def get_left_right_data(now:int=646, since:int=299)->pd.DataFrame:
    col_names = ["anr", "datum", "titel_kurz_d", "ja_proz", "links_rechts", "kons_prog"]
    dataset = pd.read_csv("../data/formatted/brj_lire_konslib_edited.csv", sep=',')
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) < now]
    dataset = dataset.replace("#ZAHL!", np.nan).replace("Nan", np.nan)
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) >= since]
    
    return dataset.drop(columns=["titel_kurz"])

print(f"Defined {get_left_right_data}")

Defined <function get_left_right_data at 0x000002340360D5E0>


In [4]:
def get_l_r_onehot(data:pd.DataFrame=get_left_right_data()["li_re"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["l3", "l2", "l1", "n_lr", "r1", "r2", "r3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_l_r_onehot}")

Defined <function get_l_r_onehot at 0x000002340DCEB670>


In [5]:
def get_kons_prog_onehot(data:pd.DataFrame=get_left_right_data()["kons_prog"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["k3", "k2", "k1", "n_kp", "p1", "p2", "p3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_kons_prog_onehot}")

Defined <function get_kons_prog_onehot at 0x000002340DCEBA60>


In [6]:
def get_rechtsform_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["rechtsform"], 5).numpy();
    result = pd.DataFrame(tensor, columns=["ref_obl", "ref_fak", "initiative", "gegen_entw", "stichfr"], index=data.index)
    
    return result.astype(int)
print(f"Defined {get_rechtsform_onehot}")

Defined <function get_rechtsform_onehot at 0x000002340DD5DF70>


In [7]:
def get_politikbereich_multihot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    polber = data[["d1e1", "d2e1", "d3e1"]]
    polber = polber.replace('.', 0)
    polber = polber.astype(int)
    
    # the names of the columns (they're a bit long)
    cols = ["Staatsordnung", "Aussenpolitik", "Sicherheitspolitik", "Wirtschaft"]
    cols += ["Landwirtschaft", "Öffentliche Finanzen", "Energie", "Verkehr und Infrastruktur"]
    cols += ["Umwelt und Lebensraum", "Sozialpolitik", "Bildung und Forschung", "Kultur, Religion, Medien"]
    
    result = pd.DataFrame(columns=cols, index = data.index)
    for i in range(len(result)):
        row = np.zeros(12)
        for p in polber.iloc[i]:
            if p != 0:
                row[p-1] = 1
        result.iloc[i] = row
    return result.astype(int)

print(f"Defined {get_politikbereich_multihot}")

Defined <function get_politikbereich_multihot at 0x000002340E259EE0>


In [8]:
def get_department_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    dep_single = data["dep"].replace('.', 2) # voting at age 18 is the only vote with a '.' and it's dep of inner
    dep_single = dep_single.astype(int)
    tensor = tf.one_hot(dep_single, 8).numpy()
    result = pd.DataFrame(tensor, columns=["EDA", "EDI", "EJPD", "VBS", "EFD", "WBF", "UVEK", "BK"], index=data.index)
    
    return result.astype(int)

print(f"Defined {get_department_onehot}")

Defined <function get_department_onehot at 0x000002340E1F2B80>


In [9]:
def get_bundesrat_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["br_pos"].replace('.', 3).astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_bund", "Dagegen_bund", "Keine_bund"], index=data.index)
    return result.astype(int)

print(f"Defined {get_bundesrat_onehot}")

Defined <function get_bundesrat_onehot at 0x000002340E1F9EE0>


In [10]:
def get_legislatur(low:int=1, high:int=10, data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    leg = data["legislatur"]
    def my_map(x:int, x_min:int=leg.min(0), x_max:int=leg.max(0), y_min:int=low, y_max:int=high)->float:
        return (x-x_min)/(x_max-x_min)*(y_max-y_min)+y_min
    
    normalized = data[["legislatur"]].applymap(my_map)
    return normalized

print(f"Defined {get_legislatur}")

Defined <function get_legislatur at 0x000002340DE27040>


In [11]:
def get_parlament_onehot(data:pd.DataFrame = get_swissvotes_data()["nr_pos"])->pd.DataFrame:
    tensor = tf.one_hot(data.astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_"+data.name, "Dagegen_"+data.name, "Keine_"+data.name], index=data.index)
    return result.astype(int)

print(f"Defined {get_parlament_onehot}")

Defined <function get_parlament_onehot at 0x000002340DED9F70>


In [12]:
def get_parties(data:pd.DataFrame = get_swissvotes_data())->list:
    import re
    
    regex_incl = re.compile("p_.*")
    regex_excl = re.compile("p_others_.*")
    
    parties_pre = list(filter(regex_incl.match, data.columns))
    parties = [p for p in parties_pre if not regex_excl.match(p)]
    return parties

print(f"Defined {get_parties}")

Defined <function get_parties at 0x000002340DE274C0>


In [13]:
def normalize_party_reco(data:pd.DataFrame = get_swissvotes_data(), names:list = get_parties())->pd.DataFrame:
    # deal with unwanted values first
    normalized = data[names].replace(".", 0)
    normalized.replace(np.nan, 0, inplace=True)
    normalized = normalized.astype(int)
    normalized.replace([3,4,5,66,9999], 0, inplace=True)
    
    result = pd.DataFrame(index=normalized.index)
    
    for p in names: # go through parties and create one hot encoding
        tensor = tf.one_hot(normalized[p], 3).numpy()
        temp = pd.DataFrame(tensor, columns=[p+"_neutral", p+"_ja", p+"_nein"], index=result.index)
        result = result.join(temp)

    return result.astype(int)
print(f"Defined {normalize_party_reco}")

Defined <function normalize_party_reco at 0x000002340DF84E50>


In [14]:
def get_vote_result(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    result = data["annahme"].replace('.', 0)
    return result.astype(int)

print(f"Defined {get_vote_result}")

Defined <function get_vote_result at 0x000002340E0178B0>


In [15]:
# creates a dataframe 
def get_canton_results(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    import re
    regex = re.compile(".*_annahme")
    canton_names = list(filter(regex.match, data.columns))
    return data[canton_names].replace('.', 0).astype(int)

print(f"Defined {get_canton_results}")

Defined <function get_canton_results at 0x000002340DF84CA0>


In [16]:
def get_volk_proz(data:pd.DataFrame=get_swissvotes_data())->pd.DataFrame:
    temp = data["volkja_proz"]
    return temp / 100

print(f"Defined {get_volk_proz}")

Defined <function get_volk_proz at 0x000002340E0D0DC0>


In [17]:
def get_canton_proz(data:pd.DataFrame=get_swissvotes_data())->pd.DataFrame:
    import re
    regex = re.compile(".._japroz")
    names = list(filter(regex.match, data.columns))
    return data[names].replace('.', 50).astype(float) / 100

print(f"Defined {get_canton_proz}")

Defined <function get_canton_proz at 0x000002340E114EE0>


## Training the net

In [18]:
# get data    
# the possible inputs for the neural net are:
    # Rechtsform (one hot),
    # Politikbereich (multi hot),
    # Department (one hot),
    # Position of the Bundesrat (one hot),
    # legislatur (normalized from 1-10),
    # Position of Nationalrat (one hot),
    # Position of Ständerat (one hot),
    # Party recommendations (one hot),
    # left, right rating (one hot),
    # conservative, liberal (one hot)
# the possible outputs are:
    # result of the votes (binary),
    # result on a canton level (binary)
    # yes percentage of the people ([0:1]),
    # yes percentage by canton ([0:1])
def get_data(input_func:list=[get_rechtsform_onehot,
                             get_politikbereich_multihot, 
                             get_department_onehot, 
                             get_bundesrat_onehot,
                             get_legislatur,
                             get_parlament_onehot,
                             get_parlament_onehot,
                             normalize_party_reco,
                             get_l_r_onehot,
                             get_kons_prog_onehot], 
            input_args:list=[None, 
                            None,
                            None,
                            None,
                            None,
                            None,
                            get_swissvotes_data()["sr_pos"],
                            None,
                            None,
                            None],
            output_func:list=[get_vote_result,
                             get_canton_results,
                             get_volk_proz, 
                             get_canton_proz],
            output_args:list=[None,
                             None,
                             None,
                             None]):
    input_list = []
    for f, a in zip(input_func, input_args):
        if type(a) == type(None):
            input_list.append(f())
        else:
            input_list.append(f(a))
    
    inputs = pd.concat(input_list, axis=1)
    
    out_list = []
    for f, a in zip(output_func, output_args):
        if type(a) == type(None):
            out_list.append(f())
        else:
            out_list.append(f(a))
    
    outputs = pd.concat(out_list, axis=1)
    
    return get_swissvotes_data(), inputs, outputs

print(f"Defined {get_data}")

Defined <function get_data at 0x000002340E154EE0>


In [19]:
def create_model(name:str, input_size:int = len(get_data()[1].columns), hidden:list=[100, 50, 20],
                 output_size:int = len(get_data(output_func=[get_vote_result,get_canton_results], 
                                                output_args=[None,None])[2].columns), 
                 activation:str="relu", activation_output:str="sigmoid", 
                 optimizer=ks.optimizers.SGD(learning_rate=0.1), 
                 loss=ks.losses.BinaryCrossentropy(), 
                 metrics:list=[ks.metrics.BinaryAccuracy(), ks.metrics.FalseNegatives()])->ks.models.Sequential:
    model = ks.models.Sequential()
    
    model.add(ks.layers.Dense(units=input_size, activation=activation, name="Input"))
    
    for i in range(len(hidden)):
        model.add(ks.layers.Dense(units=hidden[i], activation=activation, name="Hidden_"+str(i)))
        model.add(ks.layers.Dropout(rate=.1, name="Dropout_"+str(i)))
        
    model.add(ks.layers.Dense(units=output_size, activation=activation_output, name="Output"))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

print(f"Defined {create_model}")

Defined <function create_model at 0x000002340DED9550>


In [20]:
def train_model(model:ks.models.Sequential, 
                inputs:pd.DataFrame=get_data()[1], 
                outputs:pd.DataFrame=get_data(output_func=[get_vote_result,get_canton_results], 
                                              output_args=[None,None])[2], 
                test_size:float=0.6, batch_size:int=50, epochs:int=125, shuffle:bool=True)->tuple:
    
    from sklearn.model_selection import train_test_split as tss
    in_train, in_test, out_train, out_test = tss(inputs, outputs, test_size=test_size)
    
    history = model.fit(x=in_train, y=out_train, batch_size=batch_size, epochs=epochs, shuffle=shuffle)
    
    return history, in_test, out_test

print(f"Defined {train_model}")

Defined <function train_model at 0x000002340DED93A0>


### Create and Train models

There are two models. One for binary results and one for exact percentage of yes votes.

In [21]:
optimizer = ks.optimizers.RMSprop(learning_rate=0.01)
epochs = 200

model_binary = create_model("Binary Model", optimizer=optimizer)
model_prcntg = create_model("Exact Model", optimizer=optimizer, output_size=1, loss=ks.losses.MeanAbsoluteError(), 
                           metrics=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])

history_bin, in_test_bin, out_test_bin = train_model(model_binary, epochs=epochs,
                                                     outputs=get_data(output_func=[get_vote_result, get_canton_results], 
                                                                      output_args=[None, None])[2])
history_per, in_test_per, out_test_per = train_model(model_prcntg, epochs=epochs,
                                                     outputs=get_data(output_func=[get_volk_proz, get_canton_proz], 
                                                                      output_args=[None])[2])

print(model_binary.summary(), model_prcntg.summary())

Epoch 1/200

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

3/3 [==============================] - 0s 668us/step - loss: 0.7878 - binary_accuracy: 0.5047 - false_negatives: 855.0000
Epoch 2/200
3/3 [==============================] - 0s 333us/step - loss: 0.6929 - binary_accuracy: 0.5412 - false_negatives: 1068.0000
Epoch 3/200
3/3 [==============================] - 0s 667us/step - loss: 0.6833 - binary_accuracy: 0.5569 - false_negatives: 1260.0000
Epoch 4/200
3/3 [==============================] - 0s 667us/step - loss: 0.7216 - binary_accuracy: 0.596

3/3 [==============================] - 0s 667us/step - loss: 0.2402 - binary_accuracy: 0.8951 - false_negatives: 188.0000
Epoch 57/200
3/3 [==============================] - 0s 667us/step - loss: 0.1859 - binary_accuracy: 0.9244 - false_negatives: 140.0000
Epoch 58/200
3/3 [==============================] - 0s 667us/step - loss: 0.1772 - binary_accuracy: 0.9204 - false_negatives: 117.0000
Epoch 59/200
3/3 [==============================] - 0s 667us/step - loss: 0.1770 - binary_accuracy: 0.9225 - false_negatives: 140.0000
Epoch 60/200
3/3 [==============================] - 0s 667us/step - loss: 0.2043 - binary_accuracy: 0.9098 - false_negatives: 122.0000
Epoch 61/200
3/3 [==============================] - 0s 667us/step - loss: 0.2620 - binary_accuracy: 0.8798 - false_negatives: 251.0000
Epoch 62/200
3/3 [==============================] - 0s 667us/step - loss: 0.2617 - binary_accuracy: 0.8946 - false_negatives: 170.0000
Epoch 63/200
3/3 [==============================] - 0s 667us/step - 

3/3 [==============================] - 0s 334us/step - loss: 0.1489 - binary_accuracy: 0.9371 - false_negatives: 131.0000
Epoch 117/200
3/3 [==============================] - 0s 667us/step - loss: 0.1090 - binary_accuracy: 0.9515 - false_negatives: 98.0000
Epoch 118/200
3/3 [==============================] - 0s 667us/step - loss: 0.1081 - binary_accuracy: 0.9507 - false_negatives: 100.0000
Epoch 119/200
3/3 [==============================] - 0s 667us/step - loss: 0.1088 - binary_accuracy: 0.9507 - false_negatives: 83.0000
Epoch 120/200
3/3 [==============================] - 0s 667us/step - loss: 0.1006 - binary_accuracy: 0.9544 - false_negatives: 76.0000
Epoch 121/200
3/3 [==============================] - 0s 667us/step - loss: 0.0992 - binary_accuracy: 0.9585 - false_negatives: 80.0000
Epoch 122/200
3/3 [==============================] - 0s 667us/step - loss: 0.1000 - binary_accuracy: 0.9557 - false_negatives: 82.0000
Epoch 123/200
3/3 [==============================] - 0s 667us/step 

3/3 [==============================] - 0s 667us/step - loss: 0.0852 - binary_accuracy: 0.9650 - false_negatives: 68.0000
Epoch 177/200
3/3 [==============================] - 0s 667us/step - loss: 0.0670 - binary_accuracy: 0.9739 - false_negatives: 56.0000
Epoch 178/200
3/3 [==============================] - 0s 667us/step - loss: 0.0829 - binary_accuracy: 0.9661 - false_negatives: 52.0000
Epoch 179/200
3/3 [==============================] - 0s 667us/step - loss: 0.0823 - binary_accuracy: 0.9669 - false_negatives: 63.0000
Epoch 180/200
3/3 [==============================] - 0s 667us/step - loss: 0.0977 - binary_accuracy: 0.9614 - false_negatives: 68.0000
Epoch 181/200
3/3 [==============================] - 0s 334us/step - loss: 0.1022 - binary_accuracy: 0.9606 - false_negatives: 75.0000
Epoch 182/200
3/3 [==============================] - 0s 667us/step - loss: 0.0825 - binary_accuracy: 0.9645 - false_negatives: 68.0000
Epoch 183/200
3/3 [==============================] - 0s 667us/step - 

3/3 [==============================] - 0s 333us/step - loss: 0.0551 - mean_squared_error: 0.0056 - mean_absolute_error: 0.0551
Epoch 31/200
3/3 [==============================] - 0s 667us/step - loss: 0.0710 - mean_squared_error: 0.0086 - mean_absolute_error: 0.0710
Epoch 32/200
3/3 [==============================] - 0s 667us/step - loss: 0.0576 - mean_squared_error: 0.0057 - mean_absolute_error: 0.0576
Epoch 33/200
3/3 [==============================] - 0s 667us/step - loss: 0.0816 - mean_squared_error: 0.0110 - mean_absolute_error: 0.0816
Epoch 34/200
3/3 [==============================] - 0s 667us/step - loss: 0.0579 - mean_squared_error: 0.0063 - mean_absolute_error: 0.0579
Epoch 35/200
3/3 [==============================] - 0s 667us/step - loss: 0.0532 - mean_squared_error: 0.0057 - mean_absolute_error: 0.0532
Epoch 36/200
3/3 [==============================] - 0s 333us/step - loss: 0.0518 - mean_squared_error: 0.0046 - mean_absolute_error: 0.0518
Epoch 37/200
3/3 [===============

3/3 [==============================] - 0s 667us/step - loss: 0.0436 - mean_squared_error: 0.0031 - mean_absolute_error: 0.0436
Epoch 89/200
3/3 [==============================] - 0s 667us/step - loss: 0.0434 - mean_squared_error: 0.0032 - mean_absolute_error: 0.0434
Epoch 90/200
3/3 [==============================] - 0s 334us/step - loss: 0.0458 - mean_squared_error: 0.0037 - mean_absolute_error: 0.0458
Epoch 91/200
3/3 [==============================] - 0s 667us/step - loss: 0.0361 - mean_squared_error: 0.0025 - mean_absolute_error: 0.0361
Epoch 92/200
3/3 [==============================] - 0s 667us/step - loss: 0.0416 - mean_squared_error: 0.0030 - mean_absolute_error: 0.0416
Epoch 93/200
3/3 [==============================] - 0s 334us/step - loss: 0.0447 - mean_squared_error: 0.0031 - mean_absolute_error: 0.0447
Epoch 94/200
3/3 [==============================] - 0s 667us/step - loss: 0.0439 - mean_squared_error: 0.0035 - mean_absolute_error: 0.0439
Epoch 95/200
3/3 [===============

3/3 [==============================] - 0s 667us/step - loss: 0.0365 - mean_squared_error: 0.0026 - mean_absolute_error: 0.0365
Epoch 147/200
3/3 [==============================] - 0s 667us/step - loss: 0.0283 - mean_squared_error: 0.0018 - mean_absolute_error: 0.0283
Epoch 148/200
3/3 [==============================] - 0s 667us/step - loss: 0.0292 - mean_squared_error: 0.0015 - mean_absolute_error: 0.0292
Epoch 149/200
3/3 [==============================] - 0s 667us/step - loss: 0.0343 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0343
Epoch 150/200
3/3 [==============================] - 0s 667us/step - loss: 0.0316 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0316
Epoch 151/200
3/3 [==============================] - 0s 667us/step - loss: 0.0275 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0275
Epoch 152/200
3/3 [==============================] - 0s 667us/step - loss: 0.0397 - mean_squared_error: 0.0028 - mean_absolute_error: 0.0397
Epoch 153/200
3/3 [========

In [22]:
res_bin = model_binary.evaluate(x=in_test_bin, y=out_test_bin)
res_per = model_prcntg.evaluate(x=in_test_per, y=out_test_per)

print(f"Binary model results:\t{res_bin}")
print(f"Exact model results:\t{res_per}")

7/7 [==============================] - 0s 429us/step - loss: 0.0985 - mean_squared_error: 0.0149 - mean_absolute_error: 0.0985
Binary model results:	[1.4280316829681396, 0.7857764363288879, 799.0]
Exact model results:	[0.09851223230361938, 0.014882721938192844, 0.09851223230361938]


### Methods for Regression

In [23]:
import re
regex = re.compile(".*_japroz")
canton_names_proz = list(filter(regex.match, get_data()[2].columns))
regex = re.compile(".*_annahme")
canton_names_binary = list(filter(regex.match, get_data()[2].columns))

In [24]:
def binary_classifier(inputs:pd.DataFrame=get_data()[1],
                     outputs:pd.DataFrame=get_data()[2]['annahme'],
                     test_size:float=0.2, cutoff:float=0,
                     shuffle:bool=False, scale_data:bool=False,
                     visualisation:bool=True)->tuple:
    from sklearn.model_selection import train_test_split as tts
    cutoff=int(cutoff*len(inputs))
    in_train, in_test, out_train, out_test = tts(inputs[cutoff:], outputs[cutoff:], test_size=test_size, shuffle=shuffle)
    
    if(scale_data):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(in_train)
        in_train = scaler.transform(in_train)
        in_test = scaler.transform(in_test)
    
    out_pred = []
    
    from sklearn.linear_model import SGDClassifier, RidgeClassifier, RidgeClassifierCV
    from xgboost import XGBClassifier
    #from fracridge import FracRidgeRegressor, FracRidgeRegressorCV
    ridge = RidgeClassifier()
    ridge.fit(in_train,out_train)
    out_pred.append([ridge,'ridge', ridge.predict(in_test)])
    
    ridgecv = RidgeClassifierCV()
    ridgecv.fit(in_train,out_train)
    out_pred.append([ridgecv,'ridgecv', ridgecv.predict(in_test)])
    
    #fracridge = FracRidgeRegressor()
    #fracridge.fit(in_train,out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(in_test)])
    
    #fracridgecv = FracRidgeRegressorCV()
    #fracridgecv.fit(in_train,out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(in_test)])
    
    sgd = SGDClassifier(loss='log', penalty='elasticnet')
    sgd.fit(in_train, out_train)
    out_pred.append([sgd,'sgd', sgd.predict(in_test)])
    
    xgb = XGBClassifier()
    xgb.fit(in_train, out_train)
    out_pred.append([xgb,'xgb', xgb.predict(in_test)])
    
    if(visualisation):
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        from xgboost import plot_importance
        errors = []
        errors.append(['r2_score', r2_score])
        errors.append(['mse', mean_squared_error])
        errors.append(['mae', mean_absolute_error])
        
        from sklearn.metrics import plot_confusion_matrix
        for i in out_pred:
            print(i[1], ': ')
            for j in errors:
                print(j[0], j[1](i[2],out_test),' ')
            plot_confusion_matrix(i[0],in_test,out_test)
            print('\n')
    
        fig, ax = plt.subplots(len(out_pred),1, figsize=(20,30))
        axe = ax.ravel()
        for i in range(0,len(out_pred)):
            sns.regplot(ax=axe[i], x=out_pred[i][2], y=out_test, x_bins=100)
            axe[i].set_title(out_pred[i][1])
            axe[i].set_xlabel('recommendations')
            axe[i].set_ylabel('Passed')
        
        plt.rcParams["figure.figsize"] = (15, 20)
        plot_importance(xgb)
        plt.rcParams["figure.figsize"] = (6.4, 4.8)
            
    return out_pred, out_test

In [25]:
def proz_regression(inputs:pd.DataFrame=get_data()[1],
                     outputs:pd.DataFrame=get_data()[2]['volkja_proz'],
                     test_size:float=0.2, cutoff:float=0,
                     shuffle:bool=False, scale_data:bool=False,
                     visualisation:bool=True)->tuple:
    from sklearn.model_selection import train_test_split as tts
    cutoff=int(cutoff*len(inputs))
    in_train, in_test, out_train, out_test = tts(inputs[cutoff:], outputs[cutoff:], test_size=test_size, shuffle=shuffle)
        
    if(scale_data):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(in_train)
        in_train = scaler.transform(in_train)
        in_test = scaler.transform(in_test)
    
    out_pred = []
    
    from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
    from xgboost import XGBRegressor
    ridge = Ridge()
    ridge.fit(in_train,out_train)
    out_pred.append([ridge,'ridge', ridge.predict(in_test)])
    
    ridgecv = RidgeCV()
    ridgecv.fit(in_train,out_train)
    out_pred.append([ridgecv,'ridgecv', ridgecv.predict(in_test)])
    
    lasso = Lasso()
    lasso.fit(in_train,out_train)
    out_pred.append([lasso,'lasso', lasso.predict(in_test)])
    
    lassocv = LassoCV()
    lassocv.fit(in_train,out_train)
    out_pred.append([lassocv,'lassocv', lassocv.predict(in_test)])
    
    elasticnet = ElasticNet()
    elasticnet.fit(in_train,out_train)
    out_pred.append([elasticnet,'elasticnet', elasticnet.predict(in_test)])
    
    elasticnetcv = ElasticNetCV()
    elasticnetcv.fit(in_train,out_train)
    out_pred.append([elasticnetcv,'elasticnetcv', elasticnetcv.predict(in_test)])
    
    xgb = XGBRegressor()
    xgb.fit(in_train,out_train)
    out_pred.append([xgb,'xgb', xgb.predict(in_test)])
    
    if(visualisation):
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        from xgboost import plot_importance
        errors = []
        errors.append(['r2_score', r2_score])
        errors.append(['mse', mean_squared_error])
        errors.append(['mae', mean_absolute_error])
        
        for i in out_pred:
            print(i[1], ': ')
            for j in errors:
                print(j[0], j[1](i[2],out_test),' ')
            print('\n')
    
        fig, ax = plt.subplots(len(out_pred),1, figsize=(20,30))
        axe = ax.ravel()
        for i in range(0,len(out_pred)):
            sns.regplot(ax=axe[i], x=out_pred[i][2], y=out_test, x_bins=100)
            axe[i].set_title(out_pred[i][1])
            axe[i].set_xlabel('recommendations')
            axe[i].set_ylabel('Passed')
        
        plt.rcParams["figure.figsize"] = (15, 20)
        plot_importance(xgb)
        plt.rcParams["figure.figsize"] = (6.4, 4.8)
            
    return out_pred, out_test

In [26]:
def binary_classifier_iv(inputs:pd.DataFrame=get_data()[1], middle:pd.DataFrame=get_data()[2][canton_names_binary],
                  outputs:pd.DataFrame=get_data()[2]['annahme'],test_size:float=0.2, cutoff:float=0,
                  shuffle:bool=False, scale_data:bool=False,visualisation:bool=True)->tuple:
    from sklearn.model_selection import train_test_split as tts
    cutoff=int(cutoff*len(inputs))
    in_train, in_test, mid_train, mid_test = tts(inputs[cutoff:], middle[cutoff:], test_size=0.01, shuffle=shuffle)
    
    if(scale_data):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(in_train)
        in_train = scaler.transform(in_train)
        in_test = scaler.transform(in_test)
    
    mid_pred=[]
    from sklearn.linear_model import SGDClassifier, RidgeClassifier, RidgeClassifierCV
    from xgboost import XGBClassifier
    #from fracridge import FracRidgeRegressor, FracRidgeRegressorCV
    mid_ridge=[]
    for i in middle.columns:
        ridge = RidgeClassifier()
        ridge.fit(in_train,mid_train[i])
        mid_ridge.append(ridge.predict(inputs))
    mid_ridge=np.transpose(mid_ridge)
    mid_pred.append(pd.DataFrame(mid_ridge,columns=middle.columns,index=middle.index))
    
    mid_ridgecv=[]
    for i in middle.columns:
        ridgecv = RidgeClassifierCV()
        ridgecv.fit(in_train,mid_train[i])
        mid_ridgecv.append(ridgecv.predict(inputs))
    mid_ridgecv=np.transpose(mid_ridgecv)
    mid_pred.append(pd.DataFrame(mid_ridgecv,columns=middle.columns,index=middle.index))
    
    #mid_fracrdige=[]
    #for i in middle.columns:
    #    fracridge = FracRidgeRegressor()
    #    fracridge.fit(in_train,mid_train[i])
    #    mid_fracridge.append(fracridge.predict(inputs))
    #mid_fracridge=np.transpose(mid_fracridge)
    #mid_pred.append(pd.DataFrame(mid_fracridge,columns=middle.columns,index=middle.index))
    
    #mid_fracridgecv=[]
    #for i in middle.columns:
    #    fracridgecv = FracRidgeRegressorCV()
    #    fracridgecv.fit(in_train,mid_train[i])
    #    mid_fracridgecv.append(fracridgecv.predict(inputs))
    #mid_fracridgecv=np.transpose(mid_fracridgecv)
    #mid_pred.append(pd.DataFrame(mid_fracridgecv,columns=middle.columns,index=middle.index))
    
    mid_sgd=[]
    for i in middle.columns:
        sgd = SGDClassifier(loss='log', penalty='elasticnet')
        sgd.fit(in_train,mid_train[i])
        mid_sgd.append(sgd.predict(inputs))
    mid_sgd=np.transpose(mid_sgd)
    mid_pred.append(pd.DataFrame(mid_sgd,columns=middle.columns,index=middle.index))
    
    mid_xgb=[]
    for i in middle.columns:
        xgb = XGBClassifier()
        xgb.fit(in_train,mid_train[i])
        mid_xgb.append(xgb.predict(inputs))
    mid_xgb=np.transpose(mid_xgb)
    mid_pred.append(pd.DataFrame(mid_xgb,columns=middle.columns,index=middle.index))
    
    mid_train = [] 
    mid_test = []
    for i in np.arange(0,len(mid_pred)):
        mid_t1, mid_t2 = tts(mid_pred[i][cutoff:], test_size=test_size, shuffle=shuffle)
        mid_train.append(mid_t1)
        mid_test.append(mid_t2)
    out_train, out_test = tts(outputs[cutoff:],test_size=test_size, shuffle=shuffle)
    
    if(scale_data):
        for i in np.arange(0,len(mid_train)):
            scaler = StandardScaler().fit(mid_train[i])
            mid_train[i] = scaler.transform(mid_train[i])
            mid_test[i] = scaler.transform(mid_test[i])
    
    out_pred = []
    
    ridge = RidgeClassifier()
    ridge.fit(mid_train.pop(0),out_train)
    out_pred.append([ridge,'ridge', ridge.predict(mid_test.pop(0))])
    
    ridgecv = RidgeClassifierCV()
    ridgecv.fit(mid_train.pop(0),out_train)
    out_pred.append([ridgecv,'ridgecv', ridgecv.predict(mid_test.pop(0))])
    
    #fracridge = FracRidgeRegressor()
    #fracridge.fit(mid_train.pop(0),out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(mid_test.pop(0))])
    
    #fracridgecv = FracRidgeRegressorCV()
    #fracridgecv.fit(mid_train.pop(0),out_train)
    #out_pred.append([fracridge,'fracridge', fracridge.predict(mid_test.pop(0))])
    
    sgd = SGDClassifier(loss='log', penalty='elasticnet')
    sgd.fit(mid_train.pop(0), out_train)
    out_pred.append([sgd,'sgd', sgd.predict(mid_test.pop(0))])
    
    xgb = XGBClassifier()
    xgb.fit(mid_train.pop(0), out_train)
    out_pred.append([xgb,'xgb', xgb.predict(mid_test.pop(0))])
    
    if(visualisation):
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        from xgboost import plot_importance
        errors = []
        errors.append(['r2_score', r2_score])
        errors.append(['mse', mean_squared_error])
        errors.append(['mae', mean_absolute_error])
        
        from sklearn.metrics import confusion_matrix
        for i in out_pred:
            print(i[1], ': ')
            for j in errors:
                print(j[0], j[1](i[2],out_test),' ')
            confusion_matrix(i[2],out_test)
            print('\n')
    
        fig, ax = plt.subplots(len(out_pred),1, figsize=(20,30))
        axe = ax.ravel()
        for i in range(0,len(out_pred)):
            sns.regplot(ax=axe[i], x=out_pred[i][2], y=out_test, x_bins=100)
            axe[i].set_title(out_pred[i][1])
            axe[i].set_xlabel('recommendations')
            axe[i].set_ylabel('Passed')
        
        plt.rcParams["figure.figsize"] = (15, 20)
        plot_importance(xgb)
        plt.rcParams["figure.figsize"] = (6.4, 4.8)
            
    return out_pred, out_test

In [27]:
def proz_regression_iv(inputs:pd.DataFrame=get_data()[1], middle:pd.DataFrame=get_data()[2][canton_names_proz],
                  outputs:pd.DataFrame=get_data()[2]['volkja_proz'],test_size:float=0.2, cutoff:float=0,
                  shuffle:bool=False, scale_data:bool=False,visualisation:bool=True)->tuple:
    from sklearn.model_selection import train_test_split as tts
    cutoff=int(cutoff*len(inputs))
    in_train, in_test, mid_train, mid_test = tts(inputs[cutoff:], middle[cutoff:], test_size=0.01, shuffle=shuffle)
    
    if(scale_data):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler().fit(in_train)
        in_train = scaler.transform(in_train)
        in_test = scaler.transform(in_test)
    
    mid_pred=[]
    from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
    from xgboost import XGBRegressor    

    mid_ridge=[]
    for i in middle.columns:
        ridge = Ridge()
        ridge.fit(in_train,mid_train[i])
        mid_ridge.append(ridge.predict(inputs))
    mid_ridge=np.transpose(mid_ridge)
    mid_pred.append(pd.DataFrame(mid_ridge,columns=middle.columns,index=middle.index))
    
    mid_ridgecv=[]
    for i in middle.columns:
        ridgecv = RidgeCV()
        ridgecv.fit(in_train,mid_train[i])
        mid_ridgecv.append(ridgecv.predict(inputs))
    mid_ridgecv=np.transpose(mid_ridgecv)
    mid_pred.append(pd.DataFrame(mid_ridgecv,columns=middle.columns,index=middle.index))
    
    mid_lasso=[]
    for i in middle.columns:
        lasso = Lasso()
        lasso.fit(in_train,mid_train[i])
        mid_lasso.append(lasso.predict(inputs))
    mid_lasso=np.transpose(mid_lasso)
    mid_pred.append(pd.DataFrame(mid_lasso,columns=middle.columns,index=middle.index))
    
    mid_lassocv=[]
    for i in middle.columns:
        lassocv = LassoCV()
        lassocv.fit(in_train,mid_train[i])
        mid_lassocv.append(lassocv.predict(inputs))
    mid_lassocv=np.transpose(mid_lassocv)
    mid_pred.append(pd.DataFrame(mid_lassocv,columns=middle.columns,index=middle.index))
    
    mid_elasticnet=[]
    for i in middle.columns:
        elasticnet = ElasticNet()
        elasticnet.fit(in_train,mid_train[i])
        mid_elasticnet.append(elasticnet.predict(inputs))
    mid_elasticnet=np.transpose(mid_elasticnet)
    mid_pred.append(pd.DataFrame(mid_elasticnet,columns=middle.columns,index=middle.index))
    
    mid_elasticnetcv=[]
    for i in middle.columns:
        elasticnetcv = ElasticNetCV()
        elasticnetcv.fit(in_train,mid_train[i])
        mid_elasticnetcv.append(elasticnetcv.predict(inputs))
    mid_elasticnetcv=np.transpose(mid_elasticnetcv)
    mid_pred.append(pd.DataFrame(mid_elasticnetcv,columns=middle.columns,index=middle.index))
    
    mid_xgb=[]
    for i in middle.columns:
        xgb = XGBRegressor()
        xgb.fit(in_train,mid_train[i])
        mid_xgb.append(xgb.predict(inputs))
    mid_xgb=np.transpose(mid_xgb)
    mid_pred.append(pd.DataFrame(mid_xgb,columns=middle.columns,index=middle.index))
    
    mid_train = [] 
    mid_test = []
    for i in np.arange(0,len(mid_pred)):
        mid_t1, mid_t2 = tts(mid_pred[i][cutoff:], test_size=test_size, shuffle=shuffle)
        mid_train.append(mid_t1)
        mid_test.append(mid_t2)
    out_train, out_test = tts(outputs[cutoff:],test_size=test_size, shuffle=shuffle)
    
    if(scale_data):
        for i in np.arange(0,len(mid_train)):
            scaler = StandardScaler().fit(mid_train[i])
            mid_train[i] = scaler.transform(mid_train[i])
            mid_test[i] = scaler.transform(mid_test[i])
    
    out_pred = []
    
    ridge = Ridge()
    ridge.fit(mid_train.pop(0),out_train)
    out_pred.append([ridge,'ridge', ridge.predict(mid_test.pop(0))])
    
    ridgecv = RidgeCV()
    ridgecv.fit(mid_train.pop(0),out_train)
    out_pred.append([ridgecv,'ridgecv', ridgecv.predict(mid_test.pop(0))])
    
    lasso = Lasso()
    lasso.fit(mid_train.pop(0),out_train)
    out_pred.append([lasso,'lasso', lasso.predict(mid_test.pop(0))])
    
    lassocv = LassoCV()
    lassocv.fit(mid_train.pop(0),out_train)
    out_pred.append([lassocv,'lassocv', lassocv.predict(mid_test.pop(0))])
    
    elasticnet = ElasticNet()
    elasticnet.fit(mid_train.pop(0),out_train)
    out_pred.append([elasticnet,'elasticnet', elasticnet.predict(mid_test.pop(0))])
    
    elasticnetcv = ElasticNetCV()
    elasticnetcv.fit(mid_train.pop(0),out_train)
    out_pred.append([elasticnetcv,'elasticnetcv', elasticnetcv.predict(mid_test.pop(0))])
    
    xgb = XGBRegressor()
    xgb.fit(mid_train.pop(0), out_train)
    out_pred.append([xgb,'xgb', xgb.predict(mid_test.pop(0))])
    
    if(visualisation):
        from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
        from xgboost import plot_importance
        errors = []
        errors.append(['r2_score', r2_score])
        errors.append(['mse', mean_squared_error])
        errors.append(['mae', mean_absolute_error])
        
        for i in out_pred:
            print(i[1], ': ')
            for j in errors:
                print(j[0], j[1](i[2],out_test),' ')
            print('\n')
    
        fig, ax = plt.subplots(len(out_pred),1, figsize=(20,30))
        axe = ax.ravel()
        for i in range(0,len(out_pred)):
            sns.regplot(ax=axe[i], x=out_pred[i][2], y=out_test, x_bins=100)
            axe[i].set_title(out_pred[i][1])
            axe[i].set_xlabel('recommendations')
            axe[i].set_ylabel('Passed')
        
        plt.rcParams["figure.figsize"] = (15, 20)
        plot_importance(xgb)
        plt.rcParams["figure.figsize"] = (6.4, 4.8)
            
    return out_pred, out_test

In [28]:
def plots(inputs:pd.DataFrame=get_data()[1],
          outputs:pd.DataFrame=get_data()[2],
         x:str='legislatur', y:str='annahme'):
    fig, ax = plt.subplots(3,1, figsize=(20,30))
    ax[0].hist(inputs[x], bins=34)
    
    ax[1].hist(outputs[y], bins=34)
    
    ax[2]=sns.lineplot(x=inputs[x], y=outputs[y])

In [29]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

plt.hist(get_legislatur(low=1848,high=2021,data=get_swissvotes_data(since=0)),bins=34)
plt.xlabel('Year')
plt.ylabel('Votes')
plt.savefig('C:/Users/timsa/git/swiss-vote-prediction/data/figures/legislatur.eps', format='eps')

Failed to find a Ghostscript installation.  Distillation step skipped.


In [30]:
plt.hist(get_volk_proz(data=get_swissvotes_data(since=0)),bins=34)
plt.xlabel('Percentage')
plt.ylabel('Votes')
plt.savefig('C:/Users/timsa/git/swiss-vote-prediction/data/figures/proz.eps', format='eps')

Failed to find a Ghostscript installation.  Distillation step skipped.


In [31]:
proz_regression(shuffle=True, visualisation=True)

D:\IDE\anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0006950020909824994, tolerance: 0.0006802893920088105
  model = cd_fast.enet_coordinate_descent_gram(


ridge : 
r2_score 0.3799917905319141  
mse 0.015476139824971693  
mae 0.10095656962847407  


ridgecv : 
r2_score 0.4618184072743545  
mse 0.011650147943558205  
mae 0.08780479997591943  


lasso : 
r2_score -1.1166763994204645e+30  
mse 0.030969223428015273  
mae 0.14321266117833767  


lassocv : 
r2_score 0.4794355075205876  
mse 0.010911042817068457  
mae 0.0864281663581928  


elasticnet : 
r2_score -1.1166763994204645e+30  
mse 0.030969223428015273  
mae 0.14321266117833767  


elasticnetcv : 
r2_score 0.47686497271212325  
mse 0.010892889352704676  
mae 0.08642255136539763  


xgb : 
r2_score 0.4707831549818451  
mse 0.013932840772380082  
mae 0.08926102111070928  




([[Ridge(),
   'ridge',
   array([0.67810844, 0.5162435 , 0.42855464, 0.49150089, 0.53962818,
          0.58549767, 0.87131077, 0.52259791, 0.33479708, 0.36406379,
          0.7693591 , 0.6272926 , 0.48410373, 0.69958952, 0.37365385,
          0.7512204 , 0.33911228, 0.3354663 , 0.34676556, 0.25681028,
          0.67764202, 0.57669152, 0.3847361 , 0.38851593, 0.21086644,
          0.7062364 , 0.52102024, 0.8032011 , 0.35710214, 0.29614109,
          0.59494901, 0.37999953, 0.52024934, 0.39575634, 0.57475242,
          0.66042077, 0.28786545, 0.52619066, 0.7938146 , 0.3125724 ,
          0.51004148, 0.66411725, 0.41273299, 0.20237714, 0.46834729,
          0.50423142, 0.45971261, 0.42005897, 0.52014985, 0.42518093,
          0.68267464, 0.52679852, 0.51615832, 0.40675766, 0.51265431,
          0.40356346, 0.47493898, 0.36658461, 0.28780539, 0.53935011,
          0.52253692, 0.37301756, 0.52482298, 0.69068611, 0.27474397,
          0.26647315, 0.43065635, 0.7719791 , 0.48764151, 0.380245

In [32]:
binary_classifier_iv(shuffle=True)

D:\IDE\anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:28:13] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:28:13] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:28:13] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:28:13] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

D:\IDE\anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


([[RidgeClassifier(),
   'ridge',
   array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
          1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
          0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0])],
  [RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ])),
   'ridgecv',
   array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
          1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
          1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
          1, 0, 0, 1, 0])],
  [SGDClassifier(loss='log', penalty='elasticnet'),
   'sgd',
   array([1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
          1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
          1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
          1, 0, 0, 1, 1])],
  [XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylev

In [33]:
plots()